# Clasificación Avanzada

En esta guia exploraremos casos mas avanzados de clasificación: 
- Clasificación Desbalanceada: Cuando una clase esta sobre-representada en la muestra de entrenamiento.
- Clasificación Multiclase: Cuando hay mas que dos clases.

Haremos particular foco en las métricas utilizadas para estos casos.

Para esto, utilizaremos un dataset muy popular, de dígitos escritos a mano, conocido como MNIST.



## Dataset

Vamos a usar un set de datos clásico de Machine Learning, los números de MNIST. Se trata de 70 000 imágenes pequeñas de dígitos escritos a mano. El "target" de cada uno de estos dígitos es el número que representan.

Este set de datos es tan común, que en `sklearn` hay una función que permite bajarlos directamente.

In [ ]:
import numpy as np
from sklearn.datasets import fetch_openml
# Carga el dataset desde https://www.openml.org/d/554
X, y = fetch_openml(
    "mnist_784", version=1, return_X_y=True, as_frame=False, parser="pandas"
)

**Ejercicio:**
* Analice el `shape` de los features y el target, asi como el tipo de datos de estos. Mire los valores minimos y maximos que toman los "píxeles". 
* Normalice el dataset para que los features vayan de 0 a 1.

In [ ]:
# Tu turno ...

# Utilidades
La siguiente funcion le sera util para graficar los digitos:
1. Lleva cada pixel a un numero entero entre 0 y 255 (256 bit),
2. Transforma el vector en una matriz de 28 x 28. 
3. Por ultimo, grafica la matriz en escala de grises (0= blanco, 255=negro)

In [ ]:
import matplotlib.cm as cmap
import matplotlib.pyplot as plt

def plot_digit(data, subplot=plt):
    image = (data.reshape(28, 28)*255).astype(int)
    subplot.imshow(image, cmap = cmap.binary,
               interpolation="nearest")
    subplot.axis("off")

Examinemos algunos ejemplos. Iteraremos en los digitos del 0 al 8, y tomaremos el primer punto del dataset que corresponde a ese target para graficarlo:

In [ ]:
plt.figure(figsize=(10, 5))
for numero in range(0,10):
  sp = plt.subplot(2, 5, numero + 1)
  idx = (y==str(numero)).argmax()
  plot_digit(X[idx], subplot=sp)
plt.show()

## Clasificacion Binaria desbalanceada

Transformaremos este problema multiclase en un problema de clasificación binaria, en el que el objetivo es predecir si un digito es o no un cinco.

**Ejercicio**
* A continuación, cree un nuevo vector de target que consista en `1` si el target es un `'5'` o `0` si no lo es.

(Puede utilizar otro digito si lo desea)

In [ ]:
# Tu turno ...

**Ejercicio**
* Separe una 10 mil puntos al azar del dataset para utilizar como evaluación

*Nota: No es necesario separar un conjunto de validación, dado que podemos utilizar en su lugar las técnicas de validación cruzada vistas la clase anterior.*

In [ ]:
# Tu turno ...

**Ejercicio**
* Grafique la distribución de puntos de ambas clases en un histograma.
* ¿Que fraccion de los puntos pertenecen a la clase positiva? ¿Que conclusión saca de ello?

In [ ]:
# Tu turno ...

**Ejercicio**
* Instancie un regresor logístico
* Utilice técnicas de validación cruzada para estimar la exactitud (*accuracy*) sobre datos no vistos anteriormente.
* ¿Que tan bueno es el modelo? ¿Que porcentaje de aciertos alcanza?

*Nota 1: En lugar de instanciar un `LogisticRegression()`, pruebe utilizando `SGDClassifier(loss='log_loss')` del mismo modulo `sklearn.linear_model`. Este es una implementación iterativa que usa descenso por gradiente, y puede converger mas rápido.*

*Nota 2: La normalización para que los pixeles esten entre 0 y 1, en lugar de 0 y 255, si bien no era necesaria, ayuda al modelo a converger mas rápido, y le ahorra tiempo mirando la pantalla.*

In [ ]:
# Tu turno ...

**Ejercicio**
* Como benchmark, considere un clasificador naïve que siempre dice que el número _no es un cinco_ (o la clase que haya elegido). ¿Que exactitud obtiene? 
* ¿Como cambia su respuesta anterior a que tan bueno es el modelo?

*Nota: Para simplificar, puede medir la exactitud de este modelo sobre el conjunto de entrenamiento o de evaluación. La ausencia de parámetros del mismo nos asegura que no estamos sobreajustando ninguno de estos conjuntos, y la medicion es una buen estimador de su generalización*

In [ ]:
# Tu turno ...

### Matriz de Confusión y Análisis de Resultados

Para analizar los resultados, en los casos desbalanceados la exactitud no suele ser una buena métrica a monitorear. Para entender mejor la performance, podemos usar las predicciones del modelo para calcular la Matriz de Confusión, que nos dice cuantos samples de cada clase han sido clasificados en cada clase:


|     | Clase Pred 0 | Clase Pred 1 |
| -------- | ------- | ------- |
| **Clase Real 0**  | True Negative (TN) | False Possitive (FP) |
| **Clase Real 1** | False Negative (FN) | True Possitive (TP) |


Con esto, ademas de ver cada caso en particular, podemos calcular metricas que suelen ser de mucho interes:

1. Precisión: $\frac{TP}{TP + FP}$ Esta representa la fracción de elementos que predecimos como clase 1, efecivamente lo son. 
_"Cuantos de los que digo que son cinco efectivamente lo son"_

2. Exhaustividad (_recall_): $\frac{TP}{TP + FN}$ Representa la fracción de elementos de la clase 1 que hemos clasificado bien. 
_"Cuantos cincos capturamos correctamente"_

3. F1-score: $\frac{1}{F1} = \frac{1}{2}(\frac{1}{recall} + \frac{1}{precision})$ El promedio armónico entre precisión y exhaustividad. Este es un numero siempre menor al menor de ambas cantidades, por lo que mejorarlo implica mejorar ambas métricas.


**Ejercicio**
* Utilice técnicas de validación cruzada para obtener las predicciones sobre el conjunto de entrenamiento, cuando no se entrena sobre él (`sklearn.model_selection.cross_val_predict`).
* Calcule la matriz de confusión (`sklearn.metrics.confusion_matrix` )
* Calcule la precisión y exhaustividad a partir de la matriz de confusión, y compare con los valores obtenidos por las funciones implementadas en `sklearn.metrics`

*Nota:*
$$
\mathrm{precision} = \frac{TP}{TP + FP}
$$

$$
\mathrm{recall} = \frac{TP}{TP + FN}
$$

In [ ]:
# Tu turno ...

### Analisis de errores

Para analizar los errores obtenidos, es util mirar no solo el numero de equivocaciones, sino también con que confianza se equivocó. No es lo mismo clasificarlo no-cinco porque su probabilidad de ser cinco era 0.49, a porque la probabilidad era 0.0001, cuando en realidad si era cinco.

Hagamos este análisis a continuación.

**Ejercicio**
* Utilice técnicas de validación cruzada para obtener las probabilidades predichas por el modelo sobre el conjunto de entrenamiento, cuando no se entrena sobre él.

*Nota 1: `cross_val_predict` admite un argumento `method=` en el cual se puede elegir entre `predict`, `predict_proba`, `decision_function`, etc.*

In [ ]:
# Tu turno ...

**Ejercicio**

* Realice un histograma de la distribución de probabilidades.
* Plotee encimados los histogramas de estas distribuciones para los samples que tienen target 5, y para los que no. ().
* ¿Qué observa? ¿Qué conclusiones saca? ¿Cómo se podría mejorar?

*Nota 1: Considere solo la probabilidad de pertenecer a la clase 5 (segunda columna), ignorando la probabilidad de no pertenecer a ella (primera columna).*

*Nota 2: Puede realizar el `plt.hist` dos veces y se encimarán los graficos. Un `alpha=0.5` como argumento puede ayudar a ver los solapamientos, asi como un `density=True` puede ser util dado el desbalance entre clases. Experimente con el gráfico.*

In [ ]:
# Tu turno ...

Por último, examinemos los números más problemáticos. 

**Ejercicio:**
* Visualice, usando la función `plot_digits`, los 20 digitos cinco "peor clasificados", es decir aquellos que el modelo predijo que tienen la menor probabilidad de ser cinco. Para esto:
  * Copie los arrays de cincos y sus probabilidades predichas a dos variables nuevas.
  * Consiga los indices ordenados de menor a mayor probabilidad (pista: `np.argsort`)
  * Plotee los cincos dados por los 20 primeros elementos de la lista ordenada de índices.

In [ ]:
# Tu turno ...

### Análisis del modelo

Como bien sabemos, el regresor logístico ajusta un peso por cada feature, y si los features están normalizados, la magnitud de estos pesos indican la influencia del feature corespondiente en el resultado, algo conocido como _feature importance_. 

En el caso de imágenes, cada peso corresponde a un píxel. Por lo que si lo graficamos del mismo modo que hacemos a un píxel, podemos generar una suerte de _mapa de calor_ de los píxeles que mas influyen en el resultado.

*Ejercicio*
* Utilice la función `plot_digit` sobre los pesos del estimador entrenado, para visualizar la influencia de cada píxel en el resultado. Esto es de gran utilidad para entender el funcionamiento del modelo al clasificar.

* Visualice el "cinco promedio", tomando el promedio pixel a pixel sobre todos los puntos correspondientes a un cinco y visualizando el resultado con `plot_digit`. Compare este resultado con el anterior

In [ ]:
# Tu turno ...

# MultiClass

Ahora volvamos al problema original, de clasificación multiclase. 


**Ejercicio**
* Separe una 10 mil puntos al azar del dataset para utilizar como evaluación
* Grafique la distribución de puntos sobre las clases en un histograma.
* ¿Que fraccion de los puntos pertenecen a cada clase? ¿Que conclusión saca de ello?
* Instancie un regresor logístico y entrénelo sobre el conjunto de entrnamiento. ¿Cuantos parámetros tiene el modelo? ¿Que `shape` tienen los coeficientes?

*Nota 1: Por defecto, scikit-learn implementa una clasificación binaria de cada clase contra el resto, conocida como _one-vs-rest_. Al resultado de cada clasificador lo normaliza para obtener una probabilidad multiclase, en el que la suma de 1.*

In [ ]:
# Tu turno ...


**Ejercicio**
* Utilice técnicas de validación cruzada para estimar la exactitud (*accuracy*) sobre datos no vistos anteriormente.
* Utilice técnicas de validación cruzada para obtener las predicciones sobre el conjunto de entrenamiento, cuando no se entrena sobre él (`sklearn.model_selection.cross_val_predict`).
* Calcule la matriz de confusión (`sklearn.metrics.confusion_matrix` ). Examine las diferencias respecto al caso binario.
* Calcule la precisión y exhaustividad utilizando las funciones implementadas en `sklearn.metrics`. ¿Que error obtiene? ¿Porque? Pruebe utilizando el argumento `average=None`. ¿Cómo interpreta ese resultado?
* Examine el resultado de la función `classification_report` del módulo `sklearn.metrics`. 


*Nota 2: El entrenamiento ya lleva más tiempo. Si no tiene tiempo de esperar, en vez de usar validación cruzada entrene en el conjunto de entrenamiento. Puede calcular las métricas tanto sobre el conjunto de entrenamiento como en el de evaluación (al fin y al cabo, es con fines educativos ;) )*

In [ ]:
# Tu turno ...

# Formulario de Asistencia

Obligatorio completar antes del Miercoles 7 de Junio a las 23:59
[TBD]